In [1]:
from wiki_dataset import WikiDataset
from embedding_generation import EmbeddingGenerator

In [2]:
generator = EmbeddingGenerator(encoder='mpnet', version='v1', device='cuda')

In [3]:
wiki = WikiDataset(type='train', reduced=True)

In [4]:
generator.generate(wiki, batch_size=10)

Block 10/1254 done
Block 20/1254 done
Block 30/1254 done
Block 40/1254 done
Block 50/1254 done
Block 60/1254 done
Block 70/1254 done
Block 80/1254 done
Block 90/1254 done


KeyboardInterrupt: 